# Types

In Julia everything has a type, and types are hierarchical. For instance, consider the datatype `Number`:

![](https://upload.wikimedia.org/wikipedia/commons/4/40/Type-hierarchy-for-julia-numbers.png)

In [1]:
super(Number)

Any

In [2]:
subtypes(Number)

2-element Array{Any,1}:
 Complex{T<:Real}
 Real            

In [3]:
subtypes(Real)

4-element Array{Any,1}:
 AbstractFloat       
 Integer             
 Irrational{sym}     
 Rational{T<:Integer}

In [4]:
subtypes(AbstractFloat)

4-element Array{Any,1}:
 BigFloat
 Float16 
 Float32 
 Float64 

In [5]:
level = 0
function showtypetree(subtype)
    global level
    subtypelist = filter(asubtype -> asubtype != Any, subtypes(subtype))
    if length(subtypelist) > 0 
         println("\t" ^ level, subtype)        
         level += 1
         map(showtypetree, subtypelist)
         level -= 1
    else
         println("\t" ^ level, subtype)
    end    
end

showtypetree (generic function with 1 method)

In [6]:
showtypetree(Number)

Number
	Complex{T<:Real}
	Real
		AbstractFloat
			BigFloat
			Float16
			Float32
			Float64
		Integer
			BigInt
			Bool
			Signed
				Int128
				Int16
				Int32
				Int64

0

## Creating a `type`

In [7]:
abstract Feline

type Jaguar <: Feline end
type Lion <: Feline end


				Int8
			Unsigned
				UInt128
				UInt16
				UInt32
				UInt64
				UInt8
		Irrational{sym}
		Rational{T<:Integer}


In [8]:
subtypes(Feline)

2-element Array{Any,1}:
 Jaguar
 Lion  

In [9]:
showtypetree(Feline)

Feline


0

In [10]:
abstract MyAbstractType

type MyType <: MyAbstractType
   foo
   bar::Int
end

In [11]:
x = MyType("Hello World!", 10)

MyType("Hello World!",10)

In [12]:
x.foo

	Jaguar
	Lion


"Hello World!"

In [13]:
x.bar

10

In [14]:
x.foo = 3.0

3.0

In [15]:
x.foo

3.0

### Ex: British Currency

The important task is to create a **constructor function**. This has the same name as the type, and accepts three values as arguments. The special `new()` function creates a new object with the passed-in values. Remember we're still inside the type definition — this is an inner constructor.

In [16]:
type LSD
   pounds::Int 
   shillings::Int
   pence::Int
   
   function LSD(a,b,c)
    if a < 0 || b < 0 
      error("no negative numbers")
    end
    if c > 12 || b > 20
      error("too many pence or shillings")
    end
    new(a, b, c) 
   end   
end

In [17]:
price1 = LSD(5,10,6)

LSD(5,10,6)

In [18]:
price2 = LSD(1,6,8)

LSD(1,6,8)

In [19]:
fieldnames(price1)

3-element Array{Symbol,1}:
 :pounds   
 :shillings
 :pence    

The next task is to make this new type behave in the same way as other Julia objects. For example, we can't add two prices:

In [20]:
price1 + price2

LoadError: LoadError: MethodError: `+` has no method matching +(::LSD, ::LSD)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
while loading In[20], in expression starting on line 1

and the output could definitely be improved:

In [21]:
price2

LSD(1,6,8)

In [22]:
function +(a::LSD, b::LSD)
  newpence = a.pence + b.pence
  newshillings = a.shillings + b.shillings
  newpounds = a.pounds + b.pounds
  subtotal = newpence + newshillings * 12 + newpounds * 240
  (pounds, balance) = divrem(subtotal, 240)
  (shillings, pence) = divrem(balance, 12)
  LSD(pounds, shillings, pence)
end

+ (generic function with 172 methods)

In [23]:
price1 + price2

LSD(6,17,2)

The next problem to address is the unattractive presentation of LSD objects. This is fixed in exactly the same way, by adding a new method, but this time to the `show()` function, which belongs to the Base environment:

In [24]:
function Base.show(io::IO, money::LSD)
    print(io, "£$(money.pounds).$(money.shillings)s.$(money.pence)d")
end

show (generic function with 106 methods)

In [25]:
price1 + price2

£6.17s.2d

In [26]:
typealias Price LSD

LSD

In [27]:
Price(1, 19, 11)

£1.19s.11d

# Modules  

We will now import the Sundial module. See the file Sundial.jl.

In [32]:
push!(LOAD_PATH, ".")

3-element Array{ByteString,1}:
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/local/share/julia/site/v0.4"
 "/Applications/Julia-0.4.5.app/Contents/Resources/julia/share/julia/site/v0.4"      
 "."                                                                                 

In [33]:
using Sundial

The value of `latitude` is available, and can be used without the prefix, because the module exported it and because we loaded the module with `using`

In [34]:
latitude

52

In [35]:
Sundial.latitude

52

In [36]:
Sundial.latitude = 2

LoadError: LoadError: cannot assign variables in other modules
while loading In[36], in expression starting on line 1

In [37]:
Sundial.get_lat()

in Sundial.get_lat, latitude is 52


In [49]:
latitude = 40

40

In [50]:
latitude

40

In [51]:
Sundial.latitude

45

In [52]:
Main.latitude

40

In [53]:
Sundial.get_lat()

in Sundial.get_lat, latitude is 45


In [54]:
Sundial.set_lat(45)

in Sundial.set_lat, latitude is now 45


In [55]:
Sundial.get_lat()

in Sundial.get_lat, latitude is 45


The `set_lat()` function used the `global` keyword at the start of the definition. This means that code inside the function definition can change the value.

In [56]:
Sundial.latitude

45